In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# PCA
## Load data

In [ ]:
X = pd.read_csv('./leaf.csv', header=None).values
X.shape

## 1.1

In [ ]:
# scale data
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X_scaled = (X-mean) / std

# calculate covariance matrix
cov = np.cov(X_scaled.T)

# calculate eigenvectors and eigenvalues
eig_vals, eig_vecs = np.linalg.eig(cov)

# plot eigenvalues in ascending order
plt.figure(figsize=(8,4))
plt.plot(np.arange(len(eig_vals))+1,np.sort(eig_vals))
plt.xlim([1,len(eig_vals)])
plt.xlabel('eigenvalue index')
plt.ylabel('value')
plt.xticks(np.arange(len(eig_vals))+1)
plt.show()

## 1.2

In [ ]:
# selecting how many eigenvalues to use (choosing k)
eig_vals_percentage = np.flip(np.sort(eig_vals))/np.sum(eig_vals)
ks = np.arange(len(eig_vals))+1
k_sums = np.zeros_like(eig_vals)
for k, val in enumerate(eig_vals_percentage):
    k_sums[k] = k_sums[k-1]+val
    if k_sums[k]>0.95 and k_sums[k-1]<0.95:
        print(f'k={k+1} is a good choise of k')
pd.DataFrame({'k':ks, 'coverage %':k_sums}, columns= ['k', 'coverage %']).style.hide_index()

## 1.3

In [ ]:
# printing the eigenvectors for k=2
pd.DataFrame({'w1':eig_vecs[0], 'w2':eig_vecs[1]}, columns= ['w1', 'w2'])

## 1.4

In [ ]:
# calculate  and plat data on PCA space
alpha1 = X_scaled.dot(eig_vecs[0])
alpha2 = X_scaled.dot(eig_vecs[1])
plt.figure(figsize=(6,6))
plt.scatter(alpha1,alpha2)
plt.xlabel(r'$\alpha_1$')
plt.ylabel(r'$\alpha_2$')
plt.show()

## 1.5

In [ ]:
# calculate mean squared reconstruction error
X_pca = np.zeros_like(X)
for i in range(X_pca.shape[0]):
    X_pca[i] = eig_vecs[:2].T.dot(eig_vecs[:2].dot(X_scaled[i]))
MSE = np.mean((X_scaled-X_pca)**2)
print(f'The Mean squared reconstruction error is {MSE}')

# Task 1 - EM Algorithm
## Load Data

In [ ]:
data = pd.read_csv('./data.csv', header=None).values
T = data[:,:-1]
y = data[:,-1]
print(T.shape, y.shape)

In [ ]:
# initialize
c_vals = list(set(y))
n_d = len(y)
n_c = len(c_vals)
n_w = T.shape[1]
# pi = np.array([np.sum(y==c)/len(y) for c in c_vals])
# mu = np.array([[np.sum(T[:,j]*(y==c_vals[c]))/np.sum(T) for c in range(n_c)] for j in range(n_w)])
pi = np.ones(n_c) / n_c
mu = np.ones((n_w,n_c)) /n_c
gamma = np.zeros((n_d,n_c))
print(f'n_d = {n_d}, n_c = {n_c}, n_w = {n_w}')
print(f'pi.shape = {pi.shape}, mu.shape = {mu.shape}, gamma.shape = {gamma.shape}')
print(mu.sum())

# iterate EM algorithm
iters = 1
for iter in range(iters):
    # E-step
    for i in range(n_d):
        for c in range(n_c):
            gamma[i,c] = np.nan_to_num((pi[c]*np.prod(mu[:,c]**T[i,:])) / (np.sum(pi*np.prod(mu**np.tile(T[i,:],(n_c,1)).T))))
    # M-step
    for c in range(n_c):
        for j in range(n_w):
            mu[j,c] = np.sum(gamma[:,c]*T[:,j]) / np.sum(np.tile(gamma[:,c],(100,1)).T*T)
        pi[c] =  np.sum(gamma[:,c]) / n_d

# check accuracy
y_pred = [c_vals[pred] for pred in np.argmax(gamma,axis=1)]
acc = np.mean(y==y_pred)
print(f'The accuracy of the EM Algorithm is {acc}')